In [13]:
import importlib
import sys
from tqdm import tqdm
import  os
import glob
from datetime import datetime
import matplotlib.pyplot as plt
# Add the directories to the sys.path
dir_path = "C://Users//jony//Programming//Python//Anubis//anubis//" # insert your directory path
sys.path.append(dir_path + "Osiris//processing//python")
sys.path.append(dir_path + "tools")

import Analysis_tools as ATools
import proAnubis_Analysis_Tools
import Reconstruction_tools as RTools
import mplhep as hep
import Timing_tools as TTools
import rawFileReader
import Visual_tools as VTools
hep.style.use([hep.style.ATLAS])

# Specify the directory
data_list = sorted([f for f in os.listdir("data//reset") if os.path.isfile(os.path.join("data//reset", f))], reverse=True) ##all files in data directory sorted from the newest to the oldest

file_path = list(map(lambda p: dir_path+"data//reset//"+p, data_list)) #
print(file_path)

['C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240811_2240.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_2240.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_2040.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_1840.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_1640.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_1440.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_1240.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_1040.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_0840.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_0640.raw', 'C://Users//jony//Programming//Python//

In [16]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
importlib.reload(VTools)
import matplotlib.pyplot as plt

interval = 100 # Set your monitoring chunck size
order = [[0,1], [1,2], [2,3], [3,4]] # Order what you want to align
max_process_event_chunk = 3_000 # End the loop early
file_path = ['C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_0040.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240811_2240.raw']
print(file_path)
for file in file_path:
    fReader = rawFileReader.fileReader(file) # load in the classs object
    processedEvents = 0 # Initialisation
    originTime = None
    file_name = file.split("//")[-1][:-4]
    times = []
    with tqdm(total=max_process_event_chunk, desc=f"Processing Events {file_name}", unit='Events') as pbar:
        while processedEvents < max_process_event_chunk:
            processedEvents += 1
            try:
                event_chunk, tdc_met, TDC_info = fReader.get_aligned_events(order=order, interval=interval, extract_tdc_mets = True) # get the aligned events
            except Exception as e:
                print(e)
                max_process_event_chunk = processedEvents-1
                break
            
            event_time = min([event_chunk[0].tdcEvents[tdc].time for tdc in range(5) if event_chunk[0].tdcEvents[tdc].time])
            
            if originTime:
                times.append((event_time - originTime).total_seconds())
            else:
                times.append(0)
                originTime = event_time
            pbar.update(1)

    plt.figure(figsize=(10, 6))
    plt.plot(times, [i for i in range(max_process_event_chunk)], label="Time between events")
    plt.xlabel("Time (s)")
    plt.ylabel("Event Chunk Number")
    plt.title(f"Time vs events {file_name}")
    plt.show()

['C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_0040.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240811_2240.raw']


Processing Events proAnubis_240810_0040:   0%|          | 0/3000 [00:00<?, ?Events/s]

Processing Events proAnubis_240811_2240: 100%|██████████| 3000/3000 [01:01<00:00, 48.52Events/s]
